In [ ]:
#SCRIpT 1 : Simulasi Rotasi Baton, angular velocity constan
import tkinter as tk
import math

# Class Vector untuk posisi dan arah
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Tambah vektor
    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    # Kurang vektor
    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    # Kali skalar (untuk arah x panjang)
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

    # Konversi ke tuple (untuk koordinat Tkinter)
    def to_tuple(self):
        return (self.x, self.y)

# Class Baton (tongkat)
class Baton:
    def __init__(self, canvas, center, length):
        self.canvas = canvas
        self.center = center
        self.length = length
        self.angle_deg = 0                  # Sudut awal derajat
        self.angular_velocity = 2           # Kecepatan sudut derajat/frame (konstan)
        self.angular_acceleration = 0       # Percepatan sudut (0 = tetap)
        self.line = canvas.create_line(0, 0, 0, 0, width=8, fill="blue")  # Garis awal
        self.text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

    # Update posisi tongkat berdasarkan sudut
    def update(self):
        # Konversi derajat ke radian
        angle_rad = math.radians(self.angle_deg)

        # Arah rotasi menggunakan trigonometri
        direction = Vector(math.cos(angle_rad), math.sin(angle_rad))

        # Hitung ujung-ujung tongkat
        end1 = self.center + direction * self.length
        end2 = self.center - direction * self.length

        # Perbarui koordinat garis di canvas
        self.canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)

        # Tampilkan info perhitungan termasuk angular velocity & acceleration
        info = f"""Sudut (derajat): {self.angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Angular Velocity: {self.angular_velocity:.2f}°/frame
Angular Acceleration: {self.angular_acceleration:.2f}°/frame²
Ujung 1: ({end1.x:.1f}, {end1.y:.1f})
Ujung 2: ({end2.x:.1f}, {end2.y:.1f})
"""
        self.canvas.itemconfig(self.text, text=info)

        # Tambah sudut untuk animasi rotasi
        self.angle_deg = (self.angle_deg + self.angular_velocity) % 360

# Inisialisasi Tkinter dan kanvas
root = tk.Tk()
root.title("Simulasi Rotasi Baton - Nature of Code 3.1")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Titik tengah sebagai Vector
center = Vector(300, 200)

# Buat objek Baton
baton = Baton(canvas, center, length=100)

# Fungsi animasi
def animate():
    baton.update()             # Perbarui posisi dan info
    root.after(50, animate)    # Ulangi setiap 50 ms

animate()
root.mainloop()


In [ ]:
#SCRIPT 2 : Simulasi Rotasi Baton, angular_velocity = 0.05, angular_acceleration = 0.001 
import tkinter as tk
import math

# ==========================
# Class Vector2D
# ==========================
class Vector2D:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

# ==========================
# Class Baton (Tongkat)
# ==========================
class Baton:
    def __init__(self, center, length):
        self.center = center              # Titik pusat rotasi (Vector2D)
        self.length = length             # Panjang tongkat

        self.angle = 0                   # Sudut awal (radian)
        self.angular_velocity = 0.05     # Kecepatan sudut
        self.angular_acceleration = 0.0005  # Percepatan sudut

        self.line = None                 # ID garis pada canvas
        self.circle1 = None              # Ujung kiri
        self.circle2 = None              # Ujung kanan

    def update(self):
        # Rumus: angle = angle + angular velocity
        self.angle += self.angular_velocity

        # Rumus: angular velocity = angular velocity + angular acceleration
        self.angular_velocity += self.angular_acceleration

    def display(self, canvas):
        # Hitung posisi kedua ujung tongkat berdasarkan sudut
        half_len = self.length / 2
        x1 = self.center.x + math.cos(self.angle) * (-half_len)
        y1 = self.center.y + math.sin(self.angle) * (-half_len)
        x2 = self.center.x + math.cos(self.angle) * (half_len)
        y2 = self.center.y + math.sin(self.angle) * (half_len)

        if self.line is None:
            # Buat objek pertama kali
            self.line = canvas.create_line(x1, y1, x2, y2, width=4, fill="blue")
            self.circle1 = canvas.create_oval(x1-6, y1-6, x1+6, y1+6, fill="red")
            self.circle2 = canvas.create_oval(x2-6, y2-6, x2+6, y2+6, fill="red")
        else:
            # Update posisi objek menggunakan coords()
            canvas.coords(self.line, x1, y1, x2, y2)
            canvas.coords(self.circle1, x1-6, y1-6, x1+6, y1+6)
            canvas.coords(self.circle2, x2-6, y2-6, x2+6, y2+6)

        return self.angle, self.angular_velocity, self.angular_acceleration

# ==========================
# Fungsi Update Layar
# ==========================
def update():
    baton.update()
    angle, a_vel, a_acc = baton.display(canvas)

    # Update informasi hasil perhitungan ke layar
    info_text = (
        f"Angle (rad) = {angle:.3f}\n"
        f"Angular Velocity = {a_vel:.3f}\n"
        f"Angular Acceleration = {a_acc:.3f}"
    )
    label_info.config(text=info_text)

    window.after(30, update)  # Ulangi setiap 30 ms

# ==========================
# Setup Tkinter Window
# ==========================
WIDTH = 600
HEIGHT = 400
window = tk.Tk()
window.title("Simulasi Rotasi Sudut - The Nature of Code 3.2")
canvas = tk.Canvas(window, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

label_info = tk.Label(window, text="", font=("Arial", 12), justify="left")
label_info.pack()

# Buat objek Baton di tengah layar
center = Vector2D(WIDTH // 2, HEIGHT // 2)
baton = Baton(center, 150)

# Mulai animasi
update()
window.mainloop()


In [12]:
import tkinter as tk
import math
import random

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    def add(self, other):
        self.x += other.x
        self.y += other.y
    def sub(self, other):
        self.x -= other.x
        self.y -= other.y
    def mult(self, scalar):
        self.x *= scalar
        self.y *= scalar
    def div(self, scalar):
        if scalar != 0:
            self.x /= scalar
            self.y /= scalar
    def mag(self):
        return math.sqrt(self.x**2 + self.y**2)
    def normalize(self):
        m = self.mag()
        if m != 0:
            self.div(m)
    def copy(self):
        return Vector(self.x, self.y)

def constrain(val, min_val, max_val):
    return max(min_val, min(val, max_val))

class Attractor:
    def __init__(self, canvas, x=320, y=120):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.mass = 20
        self.G = 1
        # Buat objek canvas sekali
        r = self.mass * 2
        self.id = canvas.create_oval(x - r, y - r, x + r, y + r, fill="red")

    def attract(self, mover):
        force = Vector(self.position.x - mover.position.x, self.position.y - mover.position.y)
        distance = force.mag()
        distance = constrain(distance, 5, 25)
        force.normalize()
        strength = (self.G * self.mass * mover.mass) / (distance * distance)
        force.mult(strength)
        return force

    def display(self):
        # Update posisi (tidak perlu kalau posisi statis)
        r = self.mass * 2
        self.canvas.coords(self.id, self.position.x - r, self.position.y - r,
                                  self.position.x + r, self.position.y + r)

class Mover:
    def __init__(self, canvas, x, y, mass):
        self.canvas = canvas
        self.mass = mass
        self.radius = self.mass * 8
        self.position = Vector(x, y)
        self.velocity = Vector(random.uniform(-1, 1), random.uniform(-1, 1))
        self.acceleration = Vector(0, 0)

        self.angle = 0
        self.angle_velocity = 0
        self.angle_acceleration = 0

        # Buat objek canvas sekali, warna abu-abu
        r = self.radius
        self.id_circle = canvas.create_oval(x - r, y - r, x + r, y + r, fill="gray", outline="black")
        self.id_line = canvas.create_line(x, y, x + r, y, width=2, fill="black")

    def apply_force(self, force):
        f = Vector(force.x, force.y)
        f.div(self.mass)
        self.acceleration.add(f)

    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)

        self.angle_acceleration = self.acceleration.x / 10.0
        self.angle_velocity += self.angle_acceleration
        self.angle_velocity = constrain(self.angle_velocity, -0.1, 0.1)
        self.angle += self.angle_velocity

        self.acceleration.mult(0)

    def show(self):
        x, y, r = self.position.x, self.position.y, self.radius
        self.canvas.coords(self.id_circle, x - r, y - r, x + r, y + r)

        end_x = x + r * math.cos(self.angle)
        end_y = y + r * math.sin(self.angle)
        self.canvas.coords(self.id_line, x, y, end_x, end_y)

root = tk.Tk()
root.title("Example 3.2: Forces with Angular Motion - Nature of Code")

canvas = tk.Canvas(root, width=640, height=240, bg="white")
canvas.pack()

attractor = Attractor(canvas)

movers = []
for _ in range(20):
    x = random.uniform(0, 640)
    y = random.uniform(0, 240)
    mass = random.uniform(0.1, 2)
    movers.append(Mover(canvas, x, y, mass))

def animate():
    # Jangan hapus semua objek, cukup update posisi mereka
    attractor.display()
    for mover in movers:
        force = attractor.attract(mover)
        mover.apply_force(force)
        mover.update()
        mover.show()
    root.after(30, animate)

animate()
root.mainloop()


In [13]:
import tkinter as tk
import math

# Vector class untuk operasi sederhana
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    def add(self, other):
        self.x += other.x
        self.y += other.y
    def mult(self, scalar):
        self.x *= scalar
        self.y *= scalar
    def limit(self, max_val):
        mag = (self.x ** 2 + self.y ** 2) ** 0.5
        if mag > max_val:
            self.x = (self.x / mag) * max_val
            self.y = (self.y / mag) * max_val
    def copy(self):
        return Vector(self.x, self.y)
    def from_angle(angle):
        return Vector(math.cos(angle), math.sin(angle))

class CannonBall:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.r = 8
        self.topspeed = 10
        # Buat objek canvas sekali
        r = self.r
        self.id = canvas.create_oval(x - r, y - r, x + r, y + r, fill="black")

    def apply_force(self, force):
        self.acceleration.add(force)

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.topspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def display(self):
        r = self.r
        x, y = self.position.x, self.position.y
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

class Cannon:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.angle = -math.pi / 4  # -45 derajat
        self.length = 50
        self.id = None

    def draw(self):
        x, y = self.position.x, self.position.y
        # Hitung ujung cannon berdasarkan sudut
        end_x = x + self.length * math.cos(self.angle)
        end_y = y + self.length * math.sin(self.angle)
        if self.id is None:
            # Buat garis meriam pertama kali
            self.id = self.canvas.create_line(x, y, end_x, end_y, width=10, fill="gray")
        else:
            # Update posisi garis
            self.canvas.coords(self.id, x, y, end_x, end_y)

    def rotate(self, delta_angle):
        self.angle += delta_angle
        # Batasi sudut agar tetap dalam rentang wajar (optional)
        min_angle = -math.pi / 2
        max_angle = 0
        if self.angle < min_angle:
            self.angle = min_angle
        if self.angle > max_angle:
            self.angle = max_angle

    def shoot_force(self, magnitude=10):
        # Buat vektor gaya dari sudut meriam
        force = Vector(math.cos(self.angle), math.sin(self.angle))
        force.mult(magnitude)
        return force

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.2 Cannon - Nature of Code")

WIDTH, HEIGHT = 640, 360
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Inisialisasi objek cannon dan cannonball
cannon = Cannon(canvas, 50, 300)
ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
shot = False

# Fungsi untuk menangani animasi dan update posisi
def animate():
    global ball, shot

    canvas.delete("background_text")  # hapus teks info sebelumnya (optional)

    # Gambar cannon
    cannon.draw()

    if shot:
        gravity = Vector(0, 0.2)
        ball.apply_force(gravity)
        ball.update()

    ball.display()

    # Reset bola jika sudah jatuh ke bawah layar
    if ball.position.y > HEIGHT:
        ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
        shot = False

    # Tampilkan sudut meriam (dalam derajat)
    angle_deg = math.degrees(cannon.angle)
    canvas.create_text(10, 10, anchor="nw", text=f"Angle: {angle_deg:.1f}°", tag="background_text", font=("Arial", 14))

    root.after(30, animate)

# Event keypress untuk kontrol sudut dan tembak
def on_key_press(event):
    global shot
    if event.keysym == 'Left':
        cannon.rotate(-0.1)
    elif event.keysym == 'Right':
        cannon.rotate(0.1)
    elif event.keysym == 'space' and not shot:
        shot = True
        force = cannon.shoot_force(10)
        ball.apply_force(force)

root.bind("<KeyPress>", on_key_press)

animate()
root.mainloop()


In [15]:
import tkinter as tk
import math
from vector import Vector

class Vehicle:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.r = 6
        self.maxspeed = 8
        self.maxforce = 0.2
        self.id = None  # Untuk objek segitiga

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.maxspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def applyForce(self, force):
        self.acceleration.add(force)

    def seek(self, target):
        desired = target.sub(self.position)
        desired.setMag(self.maxspeed)
        steer = desired.sub(self.velocity)
        steer.limit(self.maxforce)
        self.applyForce(steer)

    def show(self):
        x = self.position.x
        y = self.position.y
        angle = self.velocity.heading()

        # Hitung titik segitiga relatif ke posisi dan sudut heading
        r = self.r
        # Titik segitiga pada arah velocity
        p1 = (x + math.cos(angle) * r * 2, y + math.sin(angle) * r * 2)
        # Titik kiri bawah
        p2 = (x + math.cos(angle + 2.5) * r * 2, y + math.sin(angle + 2.5) * r * 2)
        # Titik kanan bawah
        p3 = (x + math.cos(angle - 2.5) * r * 2, y + math.sin(angle - 2.5) * r * 2)

        points = [p1[0], p1[1], p2[0], p2[1], p3[0], p3[1]]

        if self.id is None:
            self.id = self.canvas.create_polygon(points, fill="gray", outline="black")
        else:
            self.canvas.coords(self.id, *points)

# Setup Tkinter
root = tk.Tk()
root.title("NOC 5.01 Seek - Nature of Code")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

vehicle = Vehicle(canvas, WIDTH / 2, HEIGHT / 2)

def animate():
    canvas.delete("mouse_target")
    mouse_pos = Vector(canvas.winfo_pointerx() - root.winfo_rootx(),
                       canvas.winfo_pointery() - root.winfo_rooty())
    # Gambarkan lingkaran di posisi mouse
    r = 24
    canvas.create_oval(mouse_pos.x - r, mouse_pos.y - r,
                       mouse_pos.x + r, mouse_pos.y + r,
                       fill="lightgray", outline="black", width=2, tag="mouse_target")

    # Vehicle seek posisi mouse
    vehicle.seek(mouse_pos)
    vehicle.update()
    vehicle.show()

    root.after(30, animate)

animate()
root.mainloop()
